In [1]:
%matplotlib inline

In [2]:
import theano

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 980 Ti (CNMeM is enabled with initial size: 80.0% of memory, cuDNN 5105)


In [3]:
from theano import function, config, sandbox, shared 
import theano.tensor as T

In [4]:
print( theano.config.device )
print( theano.config.lib.cnmem)  # cf. http://deeplearning.net/software/theano/library/config.html
print( theano.config.print_active_device)# Print active device at when the GPU device is initialized.

gpu
0.8
True


In [5]:
print(theano.config.allow_gc)
print(theano.config.optimizer_excluding)

False



In [46]:
import sys
sys.path.append( '../ML' )

In [47]:
from SVM import SVM, SVM_serial, SVM_parallel

In [12]:
import numpy as np
import pandas as pd

In [8]:
import os
os.getcwd()
os.listdir( os.getcwd() ) 

['2017datascibowl',
 'data_password.txt',
 'LSTM_model201702271930.save',
 'cleaning_dueSigmaFin.pyc',
 'LSTM_model201702280608.save',
 '.ipynb_checkpoints',
 'dueSigmaFinancial_kaggle.py',
 'LSTM_model.save',
 'LSTM_model201703012346.save',
 'DatSciBow2017_FullPreprocessTutorial.ipynb',
 'LSTM_model201702282350.save',
 'GRU_model201703022010.save',
 'DueSigmaFin_runs.ipynb',
 'ImagePreprocessing.ipynb',
 'dueSigmaFinancial_local.ipynb',
 'GRU_model201703012348.save',
 'GRU_model201703050709.save',
 'GRU_model201703021741.save',
 'kaggle.ipynb',
 'glass.csv',
 'DatSciBow2017_SVM.ipynb',
 '__init__.py',
 'train.h5',
 'dueSigmaFinancial_local_GRUs.ipynb',
 'cleaning_dueSigmaFin.py']

In [9]:
patients_stage1_feat = os.listdir('./2017datascibowl/stage1_feat')
print(len(patients_stage1_feat))

1595


In [10]:
patients_stage1_feat = [patientname.replace("feat_vec","") for patientname in patients_stage1_feat]

# Get the $y$ value (outcomes), the label for the class that each example belongs to, by matching patient IDs

In [13]:
y_ids = pd.read_csv('./2017datascibowl/stage1_labels.csv')
print(len(y_ids))

1397


In [14]:
y_ids_found=y_ids.loc[y_ids['id'].isin(patients_stage1_feat)]
print(len(y_ids_found))

1397


In [15]:
m = len(patients_stage1_feat)
found_indices =[]
for i in range(m):
    if patients_stage1_feat[i] in y_ids_found['id'].as_matrix():
        found_indices.append(i)

patients_stage1_feat_found = [patients_stage1_feat[i] for i in found_indices]
print(len(patients_stage1_feat_found))

1397


In [16]:
y_found=[]
for i in range(len(patients_stage1_feat_found)):
    if (patients_stage1_feat_found[i] in y_ids_found['id'].as_matrix()):
        cancer_val = y_ids_found.loc[y_ids_found['id']==patients_stage1_feat_found[i]]['cancer'].as_matrix()
        y_found.append( cancer_val )
y_found=np.array(y_found).flatten()

In [17]:
# it should be this condition, as the indices for each now correspond to each other 
len(y_found)==len(patients_stage1_feat_found)

True

In [38]:
patients_stage1_feat_found;

# Training, (Cross-)Validation, Test Set randomization and processing 

In [18]:
ratio_of_train_to_total = 0.05
ratio_valid_to_rest = 0.05
numberofexamples = len(patients_stage1_feat_found)
numberoftrainingexamples = int(numberofexamples*ratio_of_train_to_total)
numbertovalidate = int((numberofexamples - numberoftrainingexamples)*ratio_valid_to_rest)
numbertotest= numberofexamples - numberoftrainingexamples - numbertovalidate
print(numberofexamples);print(numbertotest);print(numberoftrainingexamples);print(numbertovalidate)

1397
1262
69
66


In [19]:
shuffledindices = np.random.permutation( numberofexamples)

In [35]:
#patients_train = patients_stage1_feat[shuffledindices[:numberoftrainingexamples]] 
#patients_valid = patients_stage1_feat[shuffledindices[numberoftrainingexamples:numberoftrainingexamples+numbertovalidate]]
#patients_test  = patients_stage1_feat[shuffledindices[numberoftrainingexamples+numbertovalidate:]]
#patients_train = [patients_stage1_feat[id] for id in shuffledindices[:numberoftrainingexamples]]
#patients_valid = [patients_stage1_feat[id] for id in shuffledindices[numberoftrainingexamples:numberoftrainingexamples+numbertovalidate]]
#patients_test  = [patients_stage1_feat[id] for id in shuffledindices[numberoftrainingexamples+numbertovalidate:]]
patients_train = [patients_stage1_feat_found[id] for id in shuffledindices[:numberoftrainingexamples]]
patients_valid = [patients_stage1_feat_found[id] for id in shuffledindices[numberoftrainingexamples:numberoftrainingexamples+numbertovalidate]]
patients_test  = [patients_stage1_feat_found[id] for id in shuffledindices[numberoftrainingexamples+numbertovalidate:]]

y_train = y_found[shuffledindices[:numberoftrainingexamples]]
y_valid = y_found[shuffledindices[numberoftrainingexamples:numberoftrainingexamples+numbertovalidate]]
y_test  = y_found[shuffledindices[numberoftrainingexamples+numbertovalidate:]]


Indeed

In [21]:
# sanity check
y_ids.loc[y_ids['id']== patients_train[2]]

,id,cancer
1266,e787e5fd289a9f1f6bba31569b7ad384,0


In [22]:
# sanity check
y_train[2]

0

In [23]:
#sanity check
for i in range(10,20):
    print(y_ids.loc[y_ids['id']== patients_train[i]]['cancer'].as_matrix().flatten() == y_train[i])

[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]


In [24]:
def load_feat_vec(patientid):
    f=file("./2017datascibowl/stage1_feat/"+patientid+"feat_vec","rb")
    arr = np.load(f)
    f.close()
    return arr 

In [25]:
%time patients_train_vecs = [load_feat_vec(id) for id in patients_train]
patients_train_vecs = np.array(patients_train_vecs)
print(patients_train_vecs.shape)

CPU times: user 136 ms, sys: 1.57 s, total: 1.7 s
Wall time: 9.23 s
(69, 2621448)


### Preprocess y labels to be in the -1,1 representation for the binary classes, *NOT* the 0,1 representation

In [36]:
y_train

array([1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1])

In [38]:
y_train_rep2 = np.copy(y_train)  # 2nd representation
y_train_rep2[y_train_rep2<=0]=-1

In [39]:
y_train_rep2

array([ 1, -1, -1,  1,  1, -1, -1, -1, -1,  1,  1, -1, -1, -1, -1,  1, -1,
       -1, -1,  1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1, -1,  1, -1,
        1, -1, -1, -1, -1, -1, -1, -1,  1, -1, -1, -1,  1, -1, -1,  1, -1,
        1, -1,  1, -1, -1, -1,  1, -1, -1, -1, -1, -1, -1, -1,  1, -1,  1,
        1])

In [40]:
y_train

array([1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1])

In [48]:
SVM_stage1 = SVM_parallel(patients_train_vecs,y_train_rep2,len(y_train_rep2),1.0,1.0,0.001)  # C=1.0,sigma=1.0, alpha=0.001

In [50]:
SVM_stage1.build_W();
SVM_stage1.build_update();

In [ ]:
%time SVM_stage1.train_model_full(5)  